# Jupyter Notebook to prepare files for analysis

This script prepare files for analysis which will be further used in other jupyter notebooks to evaluate aggregation plots, clusterings of the fibrils, chain labelling, and aggregation matrixes for every frame

Inputs:
Peptides concentration
Number of replica
timestep

In [5]:
concentration = 13
replica_number = 1
dt = 1000
cdt = 100000
temperature = 308
nchains = 2000
atoms_in_chain = 85

# Valuta se fargli scrivere queste info su un file in modo da farli leggere in automatico agli altri script

In [6]:
def rename_chains(filename):
    with open('fibril.pdb', 'r+') as f:
        # Read the pdb file
        lines = f.readlines()
        # Change the column number 22 which is the chain id in the pdb
        column = 22 - 1

        # We want to change all the lines of the atoms
        replace_line = list(range(4,(len(lines)-2)))

        # We start from a chain id 0 (to 9, stated below)
        # This is possibile since mdtraj does not rebuild the "broken chains"
        chain_id = 0
        count_atoms = 1
        # Delete the character for every line in range and replace with our chain id
        for l in replace_line:
            lines[l]=lines[l][0:column]+str(chain_id)+lines[l][column+1:]
            #print(count_atoms, 'count_atoms')
            count_atoms += 1
            #print(chain_id, 'word')
            #print(l, 'line')
            if count_atoms > atoms_in_chain: # as soon as we complete the first molecule
                chain_id += 1 # We add one chain identifier
                count_atoms = 1 # Reset the atom counter
                if chain_id == 10: # Only the column 22 so only one digit is allowed, therefore we start again
                    chain_id = 0

        # Delete the pdb file
        f.seek(0)

        # Replace the pdb with the new lines
        for i in lines:
            f.write(i)

### Trajectories preparation
#### Here the trajectories are appended into a single one.

In [7]:
directory = f'/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_harp2_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_{nchains}_{concentration}mM_{temperature}K_{replica_number}'
%cd $directory
%mkdir analysis
%ls

/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_harp2_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_2000_13mM_308K_1
0_analysis_prepare.ipynb         prod_2000-13mM-posre.cpt
2000_pep-13mM_1.gro              prod_2000-13mM-posre.edr
NOPBC_prod_2000-13mM-dt1000.xtc  prod_2000-13mM-posre.gro
analysis/                        prod_2000-13mM-posre.log
cg_2000-13mM.edr                 prod_2000-13mM-posre.tpr
cg_2000-13mM.gro                 prod_2000-13mM-posre.xtc
cg_2000-13mM.log                 prod_2000-13mM-posre_prev.cpt
cg_2000-13mM.tpr                 prod_2000-13mM_1.cpt
cg_2000-13mM.trr                 prod_2000-13mM_1.part0001.edr
daint_submit                     prod_2000-13mM_1.part0001.log
ff_aa-posre.mdp                  prod_2000-13mM_1.part0001.xtc
ff_aa.mdp                        prod_2000-13mM_1.part0002.edr
ff_cg.mdp                        prod_2000-13mM_1.part0002.log
ff_em.mdp                        prod_2000-13mM_1.part0002.xtc
gromos

In [3]:
import os, os.path
file_list = os.listdir('.')
file_list = list(filter(lambda k: 'part' in k, file_list))
file_list = list(filter(lambda k: 'xtc' in k, file_list))
file_list.sort()
file_list = ' '.join(file_list)
file_list

'prod_2000-13mM.part0001.xtc prod_2000-13mM.part0002.xtc prod_2000-13mM.part0003.xtc prod_2000-13mM.part0004.xtc'

In [4]:
trjcat = f'gmx_mpi trjcat -f {file_list} -o prod_{nchains}-{concentration}mM_full'
!{trjcat}

Invalid MIT-MAGIC-COOKIE-1 key                 :-) GROMACS - gmx trjcat, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                       

#### Two trajectories are made with timestep and one includes also the pbc. The one without pbc will be used for cluststize.

NOPBC trajectory

In [5]:
string = f'echo -e "0\n0\n" | gmx_mpi trjconv -s prod_{nchains}-{concentration}mM_{replica_number}.tpr -f prod_{nchains}-{concentration}mM_full.xtc -o NOPBC_prod_{nchains}-{concentration}mM-dt{dt} -dt {dt} -pbc mol -center -nice 0'
!{string}

Invalid MIT-MAGIC-COOKIE-1 key                 :-) GROMACS - gmx trjconv, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                      

Clustsize trajectory

In [6]:
string = f'echo -e "0\n0\n" | gmx_mpi trjconv -s prod_{nchains}-{concentration}mM_{replica_number}.tpr -f prod_{nchains}-{concentration}mM_full.xtc -o analysis/prod_{nchains}-{concentration}mM-dt{cdt} -dt {cdt} -nice 0'
!{string}

Invalid MIT-MAGIC-COOKIE-1 key                 :-) GROMACS - gmx trjconv, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                      

### Clustsize part
#### Writing the maxclust.ndx of the last frame for chain labelling

In [18]:
%cd $directory
%cd analysis/
string = f'ln -s ../{nchains}_pep-{concentration}mM_{replica_number}.gro'
!{string}
string = f'ln -s ../prod_{nchains}-{concentration}mM_{replica_number}.tpr'
!{string}
string = f'ln -s ../NOPBC_prod_{nchains}-{concentration}mM-dt{dt}.xtc'
!{string}

[Errno 2] No such file or directory: 'analysis/'
/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_2000_13mM_304K_analysis_test/analysis
ln: failed to create symbolic link './2000_pep-13mM.gro': File exists
ln: failed to create symbolic link './prod_2000-13mM.tpr': File exists


In [23]:
%cd $directory
%cd analysis/
# Saving the last frame coordinates for maxclust.ndx
import mdtraj as md
trajectory = md.load(f'NOPBC_prod_{nchains}-{concentration}mM-dt{dt}.xtc', top = f'{nchains}_pep-{concentration}mM_{replica_number}.gro')

print(trajectory[-1])
trajectory[-1].save('last_frame.xtc')
trajectory = 0 # I hope this will save some RAM

<mdtraj.Trajectory with 1 frames, 170000 atoms, 22000 residues, and unitcells>


In [20]:
%cd $directory
%cd analysis/
string = f'gmx_mpi clustsize -f last_frame.xtc -s prod_{nchains}-{concentration}mM_{replica_number}.tpr -mol -nice 0 -cut 0.4 -mcn last_frame.ndx'
!{string}
!{'rm *clust* histo-time.dat csize* temp.xvg'}

Invalid MIT-MAGIC-COOKIE-1 key                :-) GROMACS - gmx clustsize, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                     

In [24]:
import time
start_time = time.time()
directory = f'/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_harp2_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_{nchains}_{concentration}mM_{temperature}K_{replica_number}'
%cd $directory
%cd analysis/
string = f'gmx_mpi clustsize -f prod_{nchains}-{concentration}mM-dt{cdt}.xtc -dt {cdt} -s prod_{nchains}-{concentration}mM_{replica_number}.tpr -mol -hct matrix-dt{cdt}.dat -nice 0 -cut 0.4'
!{string}

print(time.time() - start_time)

1617812082.4155436
Invalid MIT-MAGIC-COOKIE-1 key                :-) GROMACS - gmx clustsize, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
  

In [5]:
string = f'gmx_mpi editconf -f prod_{nchains}-{concentration}mM.tpr -o fibril.pdb'
#string = f'gmx_mpi editconf -f prod_{nchains}-{concentration}mM_{replica_number}.tpr -o fibril.pdb'
!{string}

Invalid MIT-MAGIC-COOKIE-1 key                :-) GROMACS - gmx editconf, 2020.3-MODIFIED (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                      

In [4]:
rename_chains('fibril.pdb')

FileNotFoundError: [Errno 2] No such file or directory: 'fibril.pdb'